# Final Project
### Import precomp libs


In [28]:
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

In [11]:
#import the spacy english model
# "python -m spacy download en"   to download english models
import spacy
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter


In [2]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [21]:
train.head(20)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [32]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

Counter({u'(': 1,
         u')': 1,
         u'-': 2,
         u'?': 1,
         u'buy': 1,
         u'diamond': 1,
         u'koh': 1,
         u'kohinoor': 1,
         u'man': 1,
         u'noor': 1,
         u'shirt': 1,
         u'store': 1,
         u'story': 1,
         u'walk': 1})

In [33]:
len(train)

404290

In [ ]:
def make_vecs():
    vect_list = []
    results = []
    n = len(train)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = train.loc[i]
        q1, q2 = f['question1'], f['question2']
        results.append(f['is_duplicate'])
        try: 
            diff = parse(q1) - parse(q2)
            vect_list.append(diff)
        except Exception as e:
            print(e)
    return vect_list, results

vect_list, results = make_vecs()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
'float' object has no attribute 'decode'
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
'float' object has no attribute 'decode'
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000


In [41]:
import pickle 
with open('word_vecs','w') as f:
    pickle.dump(vect_list,f)
with open('results','w') as f:
    pickle.dump(results,f)

In [43]:
vect_list_p = pickle.load(open('word_vecs', 'rb'))
results_p = pickle.load(open('results', 'rb'))

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()
train = vectorizer.fit_transform(vect_list_p)
target = results_p

#### Validation

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf = MultinomialNB()
scores = cross_val_score(clf, train, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [4]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))

    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim
    
#unit test(s)
utest = train.loc[1]
print eval_sysim(utest)
utest = train.loc[3]
print eval_sysim(utest)
utest = train.loc[9]
print eval_sysim(utest)

0.934445572082
0.740894391357
0.819441244603


here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions
